In [ ]:
import os
import zipfile
import requests

# Download the dataset
url = 'hhttps://github.com/snapmoo/ML-Temporary/blob/main/FMD_Cattle.zip'
zip_file_path = 'augmented_FMD_Cattle.zip'

response = requests.get(url)
with open(zip_file_path, 'wb') as file:
    file.write(response.content)

# Extract the dataset
extract_to_path = 'dataset'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to train, validation, and test directories
train_dir = os.path.join(extract_to_path, 'train')
validation_dir = os.path.join(extract_to_path, 'validation')
test_dir = os.path.join(extract_to_path, 'test')

# Create ImageDataGenerator for training with augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='reflect'
)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Create ImageDataGenerator for test without augmentation
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 5603 images belonging to 2 classes.
Found 1887 images belonging to 2 classes.
Found 61 images belonging to 2 classes.


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.90 and logs.get('val_accuracy')>0.90):
            self.model.stop_training = True

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, regularizers, callbacks

# Load the VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.8),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=250,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
)


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/250
175/175 [==============================] - 67s 333ms/step - loss: 0.5718 - accuracy: 0.7164 - val_loss: 0.4354 - val_accuracy: 0.8093
Epoch 2/250
175/175 [==============================] - 58s 329ms/step - loss: 0.4084 - accuracy: 0.8146 - val_loss: 0.3868 - val_accuracy: 0.8400
Epoch 3/250
175/175 [==============================] - 56s 318ms/step - loss: 0.3563 - accuracy: 0.8458 - val_loss: 0.3685 - val_accuracy: 0.8534
Epoch 4/250
175/175 [==============================] - 58s 331ms/step - loss: 0.3300 - accuracy: 0.8584 - val_loss: 0.3507 - val_accuracy: 0.8556
Epoch 5/250
175/175 [==============================] - 56s 319ms/step - loss: 0.3026 - accuracy: 0.8682 - val_loss: 0.3435 - val_accuracy: 0.8772
Epoch 6/250
175/175 [==============================] - 58s 332ms/step - loss: 0.2719 - accuracy: 0.8857 - val_loss: 0.3304 - val_accuracy: 0.8648
Epoch 7/250
175/175 [==============================] - 57s 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.show()


In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('Test accuracy:', test_acc)

# Menguji semua gambar di folder test dan menampilkan hasil prediksi
import numpy as np

def predict_from_test_folder(test_generator):
    predictions = model.predict(test_generator)
    predicted_classes = np.where(predictions > 0.5, 1, 0)

    file_paths = test_generator.filepaths
    for i, file_path in enumerate(file_paths):
        print(f"File: {file_path} - Predicted: {'Terkena PMK' if predicted_classes[i] == 1 else 'Sehat'}")

predict_from_test_folder(test_generator)


In [ ]:
# Menyimpan model dalam format H5 dan JSON
model.save('model.h5')


In [ ]:
# Save the model architecture to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
import tensorflow as tf

# Load the SavedModel
saved_model_dir = '/content/saved_model'

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the TFLite model
tflite_model_path = '/content/modelv3.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
